In [7]:
from enum import Enum

class Piece(Enum):
    EMPTY = 0
    KING = 2
    DEFENDER = 1
    ATTACKER = -1

    def __str__(self):
        if self == Piece.EMPTY:
            return "."
        elif self == Piece.KING:
            return "K"
        elif self == Piece.ATTACKER:
            return "O"
        elif self == Piece.DEFENDER:
            return "X"
        else:
            raise ValueError("Invalid piece type")
        
    def get_player(self):
        if self == Piece.ATTACKER:
            return Player.ATTACKER
        elif self == Piece.DEFENDER or self == Piece.KING:
            return Player.DEFENDER
        else:
            return None
        
class Player(Enum):
    ATTACKER = 1
    DEFENDER = 2

    def __eq__(self, other):
        if isinstance(other, Player):
            return self.value == other.value
        return False

    def __str__(self):
        if self == Player.ATTACKER:
            return "Attacker"
        elif self == Player.DEFENDER:
            return "Defender"
        else:
            raise ValueError("Invalid player type")
        
    
    def __contains__(self, piece):
        if not isinstance(piece, Piece):
            return False
        
        if self == Player.DEFENDER:
            return piece == Piece.DEFENDER or piece == Piece.KING
        elif self == Player.ATTACKER:
            return piece == Piece.ATTACKER
        
        return False
        
class Coord():
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def __eq__(self, other):
        if isinstance(other, Coord):
            return self.x == other.x and self.y == other.y
        return False

class Move():
    def __init__(self, from_pos: Coord, to_pos: Coord):
        self.from_pos = from_pos
        self.to_pos = to_pos

class Game:
    board = []

    def __init__(self):
        self.board = [[Piece.EMPTY] * 13] * 13
        
    def fill_board_13_by_13(self):
        self.board = [[Piece.EMPTY for _ in range(13)] for _ in range(13)]
        # Place King at center
        self.board[6][6] = Piece.KING
        # Place defenders around the King
        defender_positions = [
            (6, 5), (6, 7), (5, 6), (7, 6),
            (6, 4), (6, 8), (4, 6), (8, 6),
            (5, 5), (5, 7), (7, 5), (7, 7)
        ]
        for x, y in defender_positions:
            self.board[x][y] = Piece.DEFENDER
        # Place attackers (example positions, adjust as needed for your rules)
        attacker_positions = [
            # Top row
            (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 6),
            # Bottom row
            (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (11, 6),
            # Left column
            (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (6, 1),
            # Right column
            (4, 12), (5, 12), (6, 12), (7, 12), (8, 12), (6, 11),
        ]
        for x, y in attacker_positions:
            self.board[x][y] = Piece.ATTACKER

    def print_board(self):
        for row in self.board:
            print(" ".join(str(piece) for piece in row))
        print()

    def find_king_position(self):
        for i, row in enumerate(self.board):
            for j, piece in enumerate(row):
                if piece == Piece.KING:
                    return Coord(i, j)
        return None

    def is_king_in_corner(self):
        king_pos = self.find_king_position()

        assert king_pos is not None, "King position not found on the board"

        x, y = king_pos.x, king_pos.y

        # Check if king is in a corner
        if (x, y) in [(0, 0), (0, 12), (12, 0), (12, 12)]:
            return True

    def is_king_surrounded(self):
        king_pos = self.find_king_position()

        assert king_pos is not None, "King position not found on the board"

        x, y = king_pos.x, king_pos.y

        # Directions: up, down, left, right
        dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        attacker_count = 0
        wall_count = 0

        for dx, dy in dirs:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 13 and 0 <= ny < 13:
                if self.board[nx][ny] == Piece.ATTACKER:
                    attacker_count += 1
            else:
                wall_count += 1

        # Surrounded by attackers on all four sides
        if attacker_count == 4:
            return True

        # Back against wall and surrounded by 3 attackers
        if wall_count == 1 and attacker_count == 3:
            return True

        return False
    
    def is_game_over(self):
        
        # Check if the king is in a corner
        if self.is_king_in_corner():
            print("Game Over: King is in a corner!")
            return True
        
        # Check if the king is surrounded
        if self.is_king_surrounded():
            print("Game Over: King is surrounded!")
            return True
        
        # Check if all pieces of one player are captured
        attacker_pieces = sum(piece == Piece.ATTACKER for row in self.board for piece in row)
        defender_pieces = sum(piece == Piece.DEFENDER or piece == Piece.KING for row in self.board for piece in row)

        if attacker_pieces == 0:
            print("Game Over: Defenders win!")
            return True
        elif defender_pieces == 0:
            print("Game Over: Attackers win!")
            return True
        
        return False
    
    def piece_at(self, pos: Coord):
        return self.board[pos.x][pos.y]
    
    # Check if a move is valid. Any piece can move to an empty space in the same row or column, as long as there are no pieces in between.
    def is_valid_move(self, from_pos: Coord, to_pos: Coord):
        assert all(0 <= element < 13 for element in [from_pos.x, from_pos.y, to_pos.x, to_pos.y]), "Coordinates must be between 0 and 12"

        if from_pos == to_pos:
            return False 
        
        piece = self.piece_at(from_pos)
        if piece == Piece.EMPTY:
            return False
        
        if self.piece_at(to_pos) != Piece.EMPTY:
            return False

        if from_pos.x == to_pos.x:
            row = self.board[from_pos.x]

            if from_pos.y < to_pos.y:
                leftmost = from_pos.y + 1
                rightmost = to_pos.y
            else:
                leftmost = to_pos.y
                # rightmost = from_pos.y - 1
                rightmost = from_pos.y 

            for piece in row[leftmost:rightmost]:
                if piece != Piece.EMPTY:
                    return False                                                                                                                                                                                              
        
        elif from_pos.y == to_pos.y:
            
            if from_pos.x < to_pos.x:
                leftmost = from_pos.x + 1
                rightmost = to_pos.x
            else:
                leftmost = to_pos.x
                # rightmost = from_pos.x - 1
                rightmost = from_pos.x

            for column in self.board[leftmost:rightmost]:
                if column[from_pos.y] != Piece.EMPTY:
                    return False   
        else:
            # Invalid move, not in the same row or column
            return False
        
        return True
    
    # Move a piece from one position to another and attack adjacent pieces if opponent's pieces are sandwiched. King can attack too.
    def move_piece_and_attack(self, from_pos: Coord, to_pos: Coord):
        piece = self.piece_at(from_pos)
        player = piece.get_player()
        self.board[to_pos.x][to_pos.y] = piece
        self.board[from_pos.x][from_pos.y] = Piece.EMPTY

        # check for attacks
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if abs(dx) + abs(dy) != 1: continue
                nx, ny = to_pos.x + dx, to_pos.y + dy
                if nx > 12 or nx < 0: continue
                if ny > 12 or ny < 0: continue

                other_piece = self.piece_at(Coord(nx, ny))
                if other_piece == Piece.EMPTY or other_piece in player or other_piece == Piece.KING:
                    continue

                nnx = to_pos.x + 2*dx
                if nnx > 12 or nnx < 0: continue

                nny = to_pos.y + 2*dy
                if nny > 12 or nny < 0: continue

                if self.piece_at(Coord(nnx, nny)) in player:
                    # We attacked a piece
                    print(f"Attacking piece at ({nx}, {ny})")
                    self.board[nx][ny] = Piece.EMPTY


In [8]:
def test_game_initialization():
    g = Game()
    assert hasattr(g, 'board')
    assert isinstance(g.board, list)
    assert hasattr(g, 'is_game_over')

def test_fill_board_13_by_13_content():
    g = Game()
    g.fill_board_13_by_13()
    assert len(g.board) == 13, "Expected board to have 13 rows"
    assert all(len(row) == 13 for row in g.board), "Expected each row to have 13 columns"
    # Check King position
    assert g.board[6][6] == Piece.KING, "Expected King to be at position (6, 6)"
    # Check that attackers and defenders are present
    attackers = sum(piece == Piece.ATTACKER for row in g.board for piece in row)
    defenders = sum(piece == Piece.DEFENDER for row in g.board for piece in row)
    assert attackers == 24, f"Expected 24 attackers on the board, got {attackers}"
    assert defenders == 12, f"Expected 12 defenders on the board, got {defenders}"

def test_print_board_runs():
    g = Game()
    g.fill_board_13_by_13()
    try:
        g.print_board()
    except Exception as e:
        assert False, f"print_board raised an exception: {e}"

def test_is_valid_move():
    g = Game()
    g.fill_board_13_by_13()

    # Valid horizontal move for defender
    from_pos = Coord(6, 4)
    to_pos = Coord(6, 2)
    assert g.is_valid_move(from_pos, to_pos) is True, "Expected valid horizontal move for defender"

    # Valid vertical move for defender
    from_pos.x, from_pos.y = 6, 4
    to_pos.x, to_pos.y = 10, 4
    assert g.is_valid_move(from_pos, to_pos) is True, "Expected valid vertical move for defender"

    # Invalid move: moving to same position
    from_pos.x, from_pos.y = 6, 5
    to_pos.x, to_pos.y = 6, 5
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid move to same position"

    # Invalid move: moving diagonally
    from_pos.x, from_pos.y = 6, 4
    to_pos.x, to_pos.y = 5, 3
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid diagonal move"

    # Invalid move: blocked by piece
    from_pos.x, from_pos.y = 6, 2
    to_pos.x, to_pos.y = 6, 8
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid move due to blocking piece"

    # Invalid move: moving to occupied position
    from_pos.x, from_pos.y = 6, 4
    to_pos.x, to_pos.y = 6, 1
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected invalid move to occupied position"

    # Invalid move: obstructed by pieces
    from_pos.x, from_pos.y = 7,12
    to_pos.x, to_pos.y = 1,12
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected obstructed move from (7,12) to (1,12)"
    
    # Invalid move: obstructed by pieces
    from_pos.x, from_pos.y = 12, 5
    to_pos.x, to_pos.y = 12, 3
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected obstructed move from (12,5) to (12,3)"
    
    # Invalid move: obstructed by pieces
    from_pos.x, from_pos.y = 6, 6
    to_pos.x, to_pos.y = 6, 5
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected obstructed move from (12,5) to (12,3)"
    
    # Invalid move: obstructed by pieces
    from_pos.x, from_pos.y = 6, 6
    to_pos.x, to_pos.y = 6, 2
    assert g.is_valid_move(from_pos, to_pos) is False, "Expected obstructed move from (12,5) to (12,3)"
    

    g.board[0][11] = Piece.KING
    from_pos.x, from_pos.y = 0, 11
    to_pos.x, to_pos.y = 0, 12
    assert g.is_valid_move(from_pos, to_pos) is True, "Expected valid move for Victory"

def test_attack_piece():
    g = Game()
    g.board[0][0] = Piece.DEFENDER
    g.board[0][1] = Piece.ATTACKER
    g.board[4][2] = Piece.DEFENDER
    from_pos = Coord(4, 2)
    to_pos = Coord(0, 2)
    
    # Move piece and attack
    g.move_piece_and_attack(from_pos, to_pos)
    
    # Check if the piece was moved
    assert g.piece_at(to_pos) == Piece.DEFENDER, "Expected piece to be moved to new position"
    assert g.piece_at(from_pos) == Piece.EMPTY, "Expected original position to be empty after move"
    
    # Check if an adjacent piece was attacked
    attacked_pos = Coord(0, 1)
    assert g.piece_at(attacked_pos) == Piece.EMPTY, "Expected attacked piece to be removed"

def test_move_piece():
    g = Game()
    g.fill_board_13_by_13()
    from_pos = Coord(6, 4)
    to_pos = Coord(6, 2)
    g.move_piece_and_attack(from_pos, to_pos)
    assert g.piece_at(to_pos) == Piece.DEFENDER, "Expected piece to be moved to new position"
    assert g.piece_at(from_pos) == Piece.EMPTY, "Expected original position to be empty after move"

def test_piece_equality():
    assert Piece.ATTACKER in Player.ATTACKER
    assert Piece.DEFENDER in Player.DEFENDER
    assert Piece.KING in Player.DEFENDER
    assert Piece.EMPTY not in Player.ATTACKER
    assert Piece.EMPTY not in Player.DEFENDER
    assert Piece.ATTACKER not in Player.DEFENDER

test_game_initialization()
test_print_board_runs()
test_fill_board_13_by_13_content()
test_is_valid_move()
test_move_piece()
test_piece_equality()

. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O . . X X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X . . . . . O
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . . . . . O . . . . . .
. . . . O O O O O . . . .



In [ ]:
import random

def find_random_move(game: Game, player: Player):
    valid_from_positions = [Coord(x, y) for x, row in enumerate(game.board) for y, piece in enumerate(row) if piece in player]
    assert len(valid_from_positions) > 0, "No valid from positions found"

    while True:
        from_pos = random.choice(valid_from_positions)

        valid_to_positions = [Coord(x, y) for x in range(13) for y in range(13) if game.is_valid_move(from_pos, Coord(x, y))]
        if len(valid_to_positions) < 1:
            continue  # No valid moves from this position, try another from_pos

        to_pos = random.choice(valid_to_positions)

        return Move(from_pos, to_pos)

def find_human_move(game: Game, player: Player):
    while True:
        try:
            from_x = int(input(f"Player {player}, enter the x-coordinate of the piece to move (0-12): "))
            from_y = int(input(f"Player {player}, enter the y-coordinate of the piece to move (0-12): "))
            to_x = int(input(f"Player {player}, enter the x-coordinate to move to (0-12): "))
            to_y = int(input(f"Player {player}, enter the y-coordinate to move to (0-12): "))

            from_pos = Coord(from_x, from_y)
            to_pos = Coord(to_x, to_y)

            if game.is_valid_move(from_pos, to_pos):
                return Move(from_pos, to_pos)
            else:
                print("Invalid move. Please try again.")
        except ValueError:
            print("Invalid input. Please enter integers between 0 and 12.")
        

def single_game():
    game = Game()
    game.fill_board_13_by_13()
    game.print_board()

    player = Player.DEFENDER  # Defender starts the game
    print(f"Starting game with player: {player}")

    while not game.is_game_over():

        if player == Player.DEFENDER:
            move = find_random_move(game, player)
            # move = find_human_move(game, player)
        else:
            move = find_random_move(game, player)

        print(f"Moving piece from {move.from_pos.x},{move.from_pos.y} to {move.to_pos.x},{move.to_pos.y}")
        game.move_piece_and_attack(move.from_pos, move.to_pos)
        game.print_board()

        player = Player.ATTACKER if player == Player.DEFENDER else Player.DEFENDER
    
    attacker_count = sum(piece == Piece.ATTACKER for row in game.board for piece in row)
    defender_count = sum(piece == Piece.DEFENDER for row in game.board for piece in row)
    print(f"Attackers left: {attacker_count}, Defenders left: {defender_count}")


single_game()

. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O . . X X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X . . . . . O
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . . . . . O . . . . . .
. . . . O O O O O . . . .

Starting game with player: Defender
Moving piece from 6,4 to 6,2
. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O X . . X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X . . . . . O
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . . . . . O . . . . . .
. . . . O O O O O . . . .

Moving piece from 11,6 to 11,11
. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O X . . X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X . .